In [31]:
import plotly
import plotly.graph_objs as go
import plotly.io as pio
from ramps import getDSR, getData
from scipy.stats import linregress

plotly.offline.init_notebook_mode(connected=True)

In [21]:
perc11 = getData("AlBSF11V.txt")
perc12 = getData("AlBSF12V.txt")
print(perc11)

[[350.0, 400.0, 450.0, 500.0, 550.0, 600.0, 650.0, 700.0, 750.0, 800.0, 850.0, 900.0, 950.0, 1000.0, 1050.0, 1100.0], [0.018975, 0.07655, 0.016685, 0.02703, 0.04541, 0.06194, 0.07015, 0.08556, 0.0732, 0.07481, 0.07327, 0.09218, 0.10484, 0.10404, 0.08126, 0.0413], [0.016565, 0.06201, 0.01307, 0.018205, 0.02368, 0.030255, 0.05293, 0.08377, 0.11138, 0.1285, 0.14754, 0.19605, 0.24775, 0.28875, 0.31065, 0.31905]]


In [22]:
trace0 = go.Scatter(
    x = perc11[0],
    y = perc11[1],
    mode = 'lines+markers',
    name = 'PERC @ 11V'
)

trace1 = go.Scatter(
    x = perc12[0],
    y = perc12[1],
    mode = 'lines+markers',
    name = 'PERC @ 12V'
)

data = [trace0, trace1]
plotly.offline.iplot(data, filename='scatter-mode')

In [8]:
def percRamps(voltage):
    return getDSR("PERC" + str(voltage) + "V.txt")

def percTrace(voltage):
    perc = percRamps(voltage)
    trace = go.Scatter(
        x = perc[0],
        y = perc[1],
        mode = 'lines+markers',
        name = 'PERC @ ' + str(voltage) + 'V'
    )
    return trace
    
data = []
for i in range(4, 13):
    data.append(percTrace(i))
    
plotly.offline.iplot(data, filename='scatter-mode')

In [11]:
def albsfRamps(voltage):
    return getDSR("AlBSF" + str(voltage) + "V.txt")

def albsfTrace(voltage):
    albsf = albsfRamps(voltage)
    trace = go.Scatter(
        x = albsf[0],
        y = albsf[1],
        mode = 'markers',
        name = 'AlBSF @ ' + str(voltage) + 'V'
    )
    return trace
    
data = []
for i in range(4, 13):
    data.append(albsfTrace(i))
    
plotly.offline.iplot(data, filename='scatter-mode')

In [37]:
volts = [v for v in range(4, 13)]
percAmps = [6.38, 13.70, 24.29, 38.3, 55.9, 76.9, 102.2, 130.9, 156.2]
albsfAmps = [5.9, 12.8, 22.8, 36.1, 52.6, 72.6, 96.5, 124.0, 154.9]

irradiances = [voltToIrr(v) for v in range(4, 13)]
percFit = linregress(irradiances, percAmps)
albsfFit = linregress(irradiances, albsfAmps)
percFit

LinregressResult(slope=0.1732021524417834, intercept=1.8680641110631626, rvalue=0.9995741189297183, pvalue=5.2464603031523385e-12, stderr=0.0019111826047914096)

In [8]:
def genLayout(title, xlabel, ylabel):
    layout = go.Layout(
        title=go.layout.Title(
            text=title,
            xref='paper',
            x=0
        ),
        xaxis=go.layout.XAxis(
            title=go.layout.xaxis.Title(
                text=xlabel
            )
        ),
        yaxis=go.layout.YAxis(
            title=go.layout.yaxis.Title(
                text=ylabel
            )
        )
    )
    return layout

In [39]:
def voltToIrr(v):
    coef = 1000 / 141.93
    return coef * (1.4*v*v - 6.8*v + 9)

trace0 = go.Scatter(
    x = [voltToIrr(v) for v in volts],
    y = percAmps,
    mode = 'lines+markers',
    name = 'PERC'
)

traceRef = go.Scatter(
    x = [e for e in range(0, 1000, 100)],
    y = [e*141.93/1000 for e in range(0, 1000, 100)],
    mode = 'lines',
    name = 'Reference'
)

trace2 = go.Scatter(
    x = irradiances,
    y = [percFit[0] * irr + percFit[1] for irr in irradiances],
    mode = 'lines',
    name = 'PERC Fit'
)

data = [trace0, trace2]
title = 'Linearity of Cells'
xlabel = 'Irradiance [W / m^2]'
ylabel = 'Short Circuit Current [mA]'
fig = go.Figure(data=data, layout=genLayout(title, xlabel, ylabel))

plotly.offline.iplot(fig, filename='blah.html')

In [41]:
trace0 = go.Scatter(
    x = [voltToIrr(v) for v in volts],
    y = albsfAmps,
    mode = 'lines+markers',
    name = 'AlBSF'
)

trace2 = go.Scatter(
    x = irradiances,
    y = [albsfFit[0] * irr + albsfFit[1] for irr in irradiances],
    mode = 'lines',
    name = 'AlBSF Fit'
)

data = [trace0, trace2]
title = 'Linearity of Cells'
xlabel = 'Irradiance [W / m^2]'
ylabel = 'Short Circuit Current [mA]'
fig = go.Figure(data=data, layout=genLayout(title, xlabel, ylabel))

plotly.offline.iplot(fig, filename='blah.html')
albsfFit

LinregressResult(slope=0.1691360627229489, intercept=0.4494946492271268, rvalue=0.99995205989651, pvalue=2.5116735343584275e-15, stderr=0.0006259898529206711)